This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

# Deep learning for timeseries

## Different kinds of timeseries tasks

## A temperature-forecasting example

In [ ]:
!wget https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip # importing the dataset
!unzip jena_climate_2009_2016.csv.zip # unzipping the dataset

**Inspecting the data of the Jena weather dataset**

In [ ]:
import os # importing the os module
fname = os.path.join("jena_climate_2009_2016.csv") # joining the file name with the dataset 

with open(fname) as f: # opening the file 
    data = f.read() # reading the file

lines = data.split("\n") # splitting the data by new line 
header = lines[0].split(",") # splitting the header by comma
lines = lines[1:] # removing the header from the data
print(header) # printing the header
print(len(lines)) # printing the length of the data

**Parsing the data**

In [ ]:
import numpy as np # importing the numpy module
temperature = np.zeros((len(lines),)) # creating a numpy array of zeros
raw_data = np.zeros((len(lines), len(header) - 1)) # creating a numpy array of zeros
for i, line in enumerate(lines): # iterating through the data
    values = [float(x) for x in line.split(",")[1:]] # splitting the data by comma and converting it to float
    temperature[i] = values[1] # assigning the temperature to the values
    raw_data[i, :] = values[:] # assigning the values to the raw data

**Plotting the temperature timeseries**

In [ ]:
from matplotlib import pyplot as plt # importing the pyplot module
plt.plot(range(len(temperature)), temperature) # plotting the temperature

**Plotting the first 10 days of the temperature timeseries**

In [ ]:
plt.plot(range(1440), temperature[:1440]) # plotting the temperature for the first 1440 data points

**Computing the number of samples we'll use for each data split**

In [ ]:
num_train_samples = int(0.5 * len(raw_data)) # calculating the number of training samples
num_val_samples = int(0.25 * len(raw_data)) # calculating the number of validation samples
num_test_samples = len(raw_data) - num_train_samples - num_val_samples # calculating the number of test samples
print("num_train_samples:", num_train_samples) # printing the number of training samples
print("num_val_samples:", num_val_samples) # printing the number of validation samples
print("num_test_samples:", num_test_samples) # printing the number of test samples

### Preparing the data

**Normalizing the data**

In [ ]:
mean = raw_data[:num_train_samples].mean(axis=0) # calculating the mean
raw_data -= mean # subtracting the mean from the raw data
std = raw_data[:num_train_samples].std(axis=0) # calculating the standard deviation
raw_data /= std # dividing the raw data by the standard deviation

In [ ]:
import numpy as np # importing the numpy module
from tensorflow import keras # importing the keras module
int_sequence = np.arange(10) # creating an integer sequence
dummy_dataset = keras.utils.timeseries_dataset_from_array( # creating a dummy dataset
    data=int_sequence[:-3], # assigning the data to the integer sequence except the last 3 elements 
    targets=int_sequence[3:], # assigning the targets to the integer sequence except the first 3 elements
    sequence_length=3, # setting the sequence length to 3
    batch_size=2, # setting the batch size to 2
)

for inputs, targets in dummy_dataset: # iterating through the dummy dataset
    for i in range(inputs.shape[0]): # iterating through the inputs shape at index 0 
        print([int(x) for x in inputs[i]], int(targets[i])) # printing the inputs and targets

**Instantiating datasets for training, validation, and testing**

In [ ]:
sampling_rate = 6 # setting the sampling rate to 6
sequence_length = 120 # setting the sequence length to 120
delay = sampling_rate * (sequence_length + 24 - 1) # calculating the delay by multiplying the sampling rate with the sequence length and 24 minus 1
batch_size = 256 # setting the batch size to 256

train_dataset = keras.utils.timeseries_dataset_from_array( # creating the training dataset 
    raw_data[:-delay], # assigning the raw data except the delay
    targets=temperature[delay:], # assigning the temperature except the delay
    sampling_rate=sampling_rate, # assigning the sampling rate
    sequence_length=sequence_length, # assigning the sequence length
    shuffle=True, # shuffling the data
    batch_size=batch_size, # assigning the batch size
    start_index=0, # setting the start index to 0
    end_index=num_train_samples) # setting the end index to the number of training samples

val_dataset = keras.utils.timeseries_dataset_from_array( # creating the validation dataset
    raw_data[:-delay], # assigning the raw data except the delay
    targets=temperature[delay:], # assigning the temperature except the delay
    sampling_rate=sampling_rate, # assigning the sampling rate
    sequence_length=sequence_length, # assigning the sequence length
    shuffle=True, # shuffling the data
    batch_size=batch_size, # assigning the batch size
    start_index=num_train_samples, # setting the start index to the number of training samples
    end_index=num_train_samples + num_val_samples) # setting the end index to the number of training samples plus the number of validation samples

test_dataset = keras.utils.timeseries_dataset_from_array( # creating the test dataset
    raw_data[:-delay], # assigning the raw data except the delay
    targets=temperature[delay:], # assigning the temperature except the delay
    sampling_rate=sampling_rate, # assigning the sampling rate
    sequence_length=sequence_length, # assigning the sequence length
    shuffle=True, # shuffling the data
    batch_size=batch_size, # assigning the batch size
    start_index=num_train_samples + num_val_samples) # setting the start index to the number of training samples plus the number of validation samples

**Inspecting the output of one of our datasets**

In [ ]:
for samples, targets in train_dataset: # iterating through the training dataset
    print("samples shape:", samples.shape) # printing the samples shape
    print("targets shape:", targets.shape) # printing the targets shape
    break # breaking the loop after the first iteration, which means only the first batch is printed

### A common-sense, non-machine-learning baseline

**Computing the common-sense baseline MAE**

In [ ]:
def evaluate_naive_method(dataset): # defining the evaluate naive method
    total_abs_err = 0. # setting the total absolute error to 0
    samples_seen = 0 # setting the samples seen to 0
    for samples, targets in dataset: # iterating through the dataset
        preds = samples[:, -1, 1] * std[1] + mean[1] # calculating the predictions
        total_abs_err += np.sum(np.abs(preds - targets)) # calculating the total absolute error
        samples_seen += samples.shape[0] # incrementing the samples seen by the shape of the samples at index 0
    return total_abs_err / samples_seen # returning the total absolute error divided by the samples seen

print(f"Validation MAE: {evaluate_naive_method(val_dataset):.2f}") # printing the validation MAE 
print(f"Test MAE: {evaluate_naive_method(test_dataset):.2f}") # printing the test MAE

### Let's try a basic machine-learning model

**Training and evaluating a densely connected model**

In [ ]:
from tensorflow import keras # importing the keras module
from tensorflow.keras import layers # importing the layers module

inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1])) # creating the input layer with the shape of the sequence length and the raw data shape at index -1
x = layers.Flatten()(inputs) # flattening the inputs
x = layers.Dense(16, activation="relu")(x) # adding a dense layer with 16 units and relu activation
outputs = layers.Dense(1)(x) # adding a dense layer with 1 unit
model = keras.Model(inputs, outputs) # creating the model

callbacks = [ # creating the callbacks
    keras.callbacks.ModelCheckpoint("jena_dense.keras", # saving the model to jena_dense.keras
                                    save_best_only=True) # saving the best model only based on the validation loss
]
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"]) # compiling the model with rmsprop optimizer, mse loss, and mae metric 
history = model.fit(train_dataset, # training the model
                    epochs=10, # setting the number of epochs to 10
                    validation_data=val_dataset, # setting the validation data
                    callbacks=callbacks) # setting the callbacks

model = keras.models.load_model("jena_dense.keras") # loading the model
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}") # printing the test MAE

**Plotting results**

In [ ]:
import matplotlib.pyplot as plt # importing the pyplot module
loss = history.history["mae"] # getting the MAE
val_loss = history.history["val_mae"] # getting the validation MAE
epochs = range(1, len(loss) + 1) # getting the epochs by adding 1 to the length of the loss 
plt.figure() # creating a figure
plt.plot(epochs, loss, "bo", label="Training MAE") # plotting the training MAE
plt.plot(epochs, val_loss, "b", label="Validation MAE") # plotting the validation MAE
plt.title("Training and validation MAE") # setting the title
plt.legend() # adding the legend
plt.show() # showing the plot

### Let's try a 1D convolutional model

In [ ]:
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1])) # creating the input layer with the shape of the sequence length and the raw data shape at index -1
x = layers.Conv1D(8, 24, activation="relu")(inputs) # adding a 1D convolutional layer with 8 filters, 24 kernel size, and relu activation
x = layers.MaxPooling1D(2)(x) # adding a 1D max pooling layer with 2 pool size
x = layers.Conv1D(8, 12, activation="relu")(x) # adding a 1D convolutional layer with 8 filters, 12 kernel size, and relu activation
x = layers.MaxPooling1D(2)(x) # adding a 1D max pooling layer with 2 pool size
x = layers.Conv1D(8, 6, activation="relu")(x) # adding a 1D convolutional layer with 8 filters, 6 kernel size, and relu activation
x = layers.GlobalAveragePooling1D()(x) # adding a global average pooling layer
outputs = layers.Dense(1)(x) # adding a dense layer with 1 unit
model = keras.Model(inputs, outputs) # creating the model

callbacks = [ # creating the callbacks
    keras.callbacks.ModelCheckpoint("jena_conv.keras", # saving the model to jena_conv.keras
                                    save_best_only=True) # saving the best model only based on the validation loss
]
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"]) # compiling the model with rmsprop optimizer, mse loss, and mae metric
history = model.fit(train_dataset, # training the model
                    epochs=10, # setting the number of epochs to 10
                    validation_data=val_dataset, # setting the validation data
                    callbacks=callbacks) # setting the callbacks

model = keras.models.load_model("jena_conv.keras") # loading the model
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}") # printing the test MAE

### A first recurrent baseline

**A simple LSTM-based model**

In [ ]:
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1])) # creating the input layer with the shape of the sequence length and the raw data shape at index -1
x = layers.LSTM(16)(inputs) # adding an LSTM layer with 16 units 
outputs = layers.Dense(1)(x) # adding a dense layer with 1 unit    
model = keras.Model(inputs, outputs) # creating the model

callbacks = [ # creating the callbacks
    keras.callbacks.ModelCheckpoint("jena_lstm.keras", # saving the model to jena_lstm.keras
                                    save_best_only=True) # saving the best model only based on the validation loss
]
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"]) # compiling the model with rmsprop optimizer, mse loss, and mae metric
history = model.fit(train_dataset, # training the model
                    epochs=10, # setting the number of epochs to 10
                    validation_data=val_dataset, # setting the validation data
                    callbacks=callbacks) # setting the callbacks

model = keras.models.load_model("jena_lstm.keras") # loading the model
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}") # printing the test MAE

## Understanding recurrent neural networks

**NumPy implementation of a simple RNN**

In [ ]:
import numpy as np # importing the numpy module
timesteps = 100 # setting the timesteps to 100
input_features = 32 # setting the input features to 32
output_features = 64 # setting the output features to 64
inputs = np.random.random((timesteps, input_features)) # creating random inputs based on the timesteps and input features
state_t = np.zeros((output_features,)) # creating a zero state based on the output features
W = np.random.random((output_features, input_features)) # creating random weights based on the output features and input features
U = np.random.random((output_features, output_features)) # creating random weights based on the output features and output features
b = np.random.random((output_features,)) # creating random biases based on the output features
successive_outputs = [] # creating an empty list
for input_t in inputs: # iterating through the inputs
    output_t = np.tanh(np.dot(W, input_t) + np.dot(U, state_t) + b) # calculating the output based on the input, state, and biases by using the tanh activation function
    successive_outputs.append(output_t) # appending the output to the successive outputs
    state_t = output_t # updating the state to the output 
final_output_sequence = np.stack(successive_outputs, axis=0) # stacking the successive outputs along the 0 axis 

### A recurrent layer in Keras

**An RNN layer that can process sequences of any length**

In [ ]:
num_features = 14 # setting the number of features to 14
inputs = keras.Input(shape=(None, num_features)) # creating the input layer with the shape of None and the number of features
outputs = layers.SimpleRNN(16)(inputs) # adding a simple RNN layer with 16 units

**An RNN layer that returns only its last output step**

In [ ]:
num_features = 14 # setting the number of features to 14
steps = 120 # setting the steps to 120
inputs = keras.Input(shape=(steps, num_features)) # creating the input layer with the shape of steps and the number of features
outputs = layers.SimpleRNN(16, return_sequences=False)(inputs) # adding a simple RNN layer with 16 units and return sequences set to False
print(outputs.shape) # printing the outputs shape

**An RNN layer that returns its full output sequence**

In [ ]:
num_features = 14 # setting the number of features to 14
steps = 120 # setting the steps to 120
inputs = keras.Input(shape=(steps, num_features)) # creating the input layer with the shape of steps and the number of features
outputs = layers.SimpleRNN(16, return_sequences=True)(inputs) # adding a simple RNN layer with 16 units and return sequences set to True
print(outputs.shape) # printing the outputs shape

**Stacking RNN layers**

In [ ]:
inputs = keras.Input(shape=(steps, num_features)) # creating the input layer with the shape of steps and the number of features
x = layers.SimpleRNN(16, return_sequences=True)(inputs) # adding a simple RNN layer with 16 units and return sequences set to True
x = layers.SimpleRNN(16, return_sequences=True)(x) # adding a simple RNN layer with 16 units and return sequences set to True
outputs = layers.SimpleRNN(16)(x) # adding a simple RNN layer with 16 units

## Advanced use of recurrent neural networks

### Using recurrent dropout to fight overfitting

**Training and evaluating a dropout-regularized LSTM**

In [ ]:
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1])) # creating the input layer with the shape of the sequence length and the raw data shape at index -1
x = layers.LSTM(32, recurrent_dropout=0.25)(inputs) # adding an LSTM layer with 32 units and recurrent dropout set to 0.25
x = layers.Dropout(0.5)(x) # adding a dropout layer with 0.5 dropout rate
outputs = layers.Dense(1)(x) # adding a dense layer with 1 unit
model = keras.Model(inputs, outputs) # creating the model

callbacks = [ # creating the callbacks
    keras.callbacks.ModelCheckpoint("jena_lstm_dropout.keras", # saving the model to jena_lstm_dropout.keras
                                    save_best_only=True) # saving the best model only based on the validation loss
]
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"]) # compiling the model with rmsprop optimizer, mse loss, and mae metric
history = model.fit(train_dataset, # training the model
                    epochs=50, # setting the number of epochs to 50
                    validation_data=val_dataset, # setting the validation data
                    callbacks=callbacks) # setting the callbacks

In [ ]:
inputs = keras.Input(shape=(sequence_length, num_features)) # creating the input layer with the shape of the sequence length and the number of features
x = layers.LSTM(32, recurrent_dropout=0.2, unroll=True)(inputs) # adding an LSTM layer with 32 units, recurrent dropout set to 0.2, and unroll set to True

### Stacking recurrent layers

**Training and evaluating a dropout-regularized, stacked GRU model**

In [ ]:
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1])) # creating the input layer with the shape of the sequence length and the raw data shape at index -1
x = layers.GRU(32, recurrent_dropout=0.5, return_sequences=True)(inputs) # adding a GRU layer with 32 units, recurrent dropout set to 0.5, and return sequences set to True
x = layers.GRU(32, recurrent_dropout=0.5)(x) # adding a GRU layer with 32 units and recurrent dropout set to 0.5
x = layers.Dropout(0.5)(x) # adding a dropout layer with 0.5 dropout rate
outputs = layers.Dense(1)(x) # adding a dense layer with 1 unit
model = keras.Model(inputs, outputs) # creating the model

callbacks = [ # creating the callbacks
    keras.callbacks.ModelCheckpoint("jena_stacked_gru_dropout.keras", # saving the model to jena_stacked_gru_dropout.keras
                                    save_best_only=True) # saving the best model only based on the validation loss
]
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"]) # compiling the model with rmsprop optimizer, mse loss, and mae metric
history = model.fit(train_dataset, # training the model
                    epochs=50, # setting the number of epochs to 50
                    validation_data=val_dataset, # setting the validation data
                    callbacks=callbacks) # setting the callbacks
model = keras.models.load_model("jena_stacked_gru_dropout.keras") # loading the model
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}") # printing the test MAE

### Using bidirectional RNNs

**Training and evaluating a bidirectional LSTM**

In [ ]:
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1])) # creating the input layer with the shape of the sequence length and the raw data shape at index -1
x = layers.Bidirectional(layers.LSTM(16))(inputs) # adding a bidirectional LSTM layer with 16 units
outputs = layers.Dense(1)(x) # adding a dense layer with 1 unit
model = keras.Model(inputs, outputs) # creating the model

model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"]) # compiling the model with rmsprop optimizer, mse loss, and mae metric
history = model.fit(train_dataset, # training the model
                    epochs=10, # setting the number of epochs to 10
                    validation_data=val_dataset) # setting the validation data

### Going even further

## Summary